#**Training YOLO model in Google Colab**

#Verifying NVIDIA GPU availability

In [ ]:
!nvidia-smi

We'll upload the dataset directly through colab, split it into training and validation sets, and create the config file needed to train the YOLO model.

In [1]:
# unzipping to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics requires a particular folder structure to store training data for models. The root folder is named `data`. Inside, there are two main folders:

* `Train`: These are the actual images used to train the model. In one epoch of training, every image in the train set is passed into the neural network. The training algorithm adjusts the network weights to fit the data in the images.

* `Validation`: These images are used to check the model's performance at the end of each training epoch.

In each of these folders is an `images` folder and a `labels` folder, which hold the image files and annotation files respectively.

I wrote a [Python script](https://github.com/iamvenk00/Anti-Cheating-Mechanism/blob/main/train_val_split.py) `train_val_split.py` that will automatically create the required folder structure and randomly move `75%` of dataset to the `train` folder and `25%` to the `validation` folder. Run the following code block to download and execute the script.

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/iamvenk00/Anti-Cheating-Mechanism/refs/heads/main/train_val_split.py

!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.75

#Installing Ultralytics

In [ ]:
!pip install ultralytics

#Run the code block below to automatically generate a `data.yaml` configuration file.

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

There are several YOLO11 models sizes available to train, including `yolo11n.pt`, `yolo11s.pt`, `yolo11m.pt`, `yolo11l.pt`, and `yolo11xl.pt`. Larger models run slower but have higher accuracy, while smaller models run faster but have lower accuracy. If you aren't sure which model size to use, `yolo11s.pt` is a good starting point.

You can also train YOLOv8 or YOLOv5 models by substituting `yolo11` for `yolov8` or `yolov5`.

Setting the number of epochs dictates how long the model will train for. The best amount of epochs to use depends on the size of the dataset and the model architecture. If your dataset has less than 200 images, a good starting point is 60 epochs. If your dataset has more than 200 images, a good starting point is 40 epochs.

Resolution has a large impact on the speed and accuracy of the model: a lower resolution model will have higher speed but less accuracy. YOLO models are typically trained and inferenced at a 640x640 resolution.

#Run the following code block to begin training. If you want to use a different model, number of epochs or resolution, change `model`, `epochs`, `imgsz`.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11l.pt epochs=65 imgsz=640

The training algorithm will parse the images in the `training` and `validation` directories and then start training the model. At the end of each training epoch, the program runs the model on the validation dataset and reports the resulting mAP, precision and recall. As training continues, the mAP should generally increase with each epoch. Training will end once it goes through the number of epochs specified by `epochs`.

The best trained model weights will be saved in `content/runs/detect/train/weights/best.pt`. Additional information about training is saved in the `content/runs/detect/train` folder, including a `results.png` file that shows how loss, precision, recall and mAP progressed over each epoch.

#The model has been trained.

The commands below run the model on the images in the validation folder and then display the results for the first 10 images.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

The model should draw a box around each object of interest in each image.

You can also run the model on video files or other images images by uploading them to this notebook and using the above `!yolo detect predict` command, where `source` points to the location of the video file, image, or folder of images.

The results will be saved in `runs/detect/predict`.

#Download the custom YOLO model

Zip and download the trained model by running the code blocks below.

The code creates a folder named `my_model`, moves the model weights into it, and renames them from `best.pt` to `my_model.pt`. It also adds the training results in case you want to refer to them later. It then zips the folder as `my_model.zip`.

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Download. You can also download the model directly from the sidebar.
from google.colab import files
files.download('/content/my_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Now, we'll run our downloaded model on a local device.

I wrote a [Python script](https://github.com/iamvenk00/Anti-Cheating-Mechanism/blob/main/yolo_detect.py), `yolo_detect.py`, that shows how to load a model, run inference on an image source, parse the inference results and display boxes around each detected class in the image. The script gives an example of how to work with Ultralytics YOLO models in Python.

#Deploying on PC

Install Ultralytics (which also installs import libraries like OpenCV-Python, Numpy, and PyTorch) by issuing the following command:
```
pip install ultralytics
```

If you have an NVIDIA GPU, you can install the GPU-enabled version of PyTorch by issuing the following command:
```
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

Extract downloaded model
Take the `my_model.zip` file you downloaded and unzip it to a folder on your PC. In the terminal, move into the unzipped folder using:
```
cd path/to/folder
```

Download the `yolo_detect.py` script into the `my_model` folder using:
```
curl -o yolo_detect.py https://raw.githubusercontent.com/iamvenk00/Anti-Cheating-Mechanism/refs/heads/main/yolo_detect.py
```

We're now ready to run the script. To run inference, issue:
```
python yolo_detect.py --model my_model.pt --source usb0
```

A window will appear showing a live feed from your webcam with boxes drawn around detected objects in each frame.

You can also run the model on an video file, image, or folder of images. To see a full list of arguments for `yolo_detect.py`, see the [file](https://github.com/iamvenk00/Anti-Cheating-Mechanism/blob/main/arguments.txt).